In [1]:
#необходимые пакеты 
import ee, geemap, os
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import geopandas as gpd 
import scipy

In [2]:
ee.Initialize()

In [5]:
region_boundary = geemap.geojson_to_ee('data/budenovsk_district_boundaries.geojson')

